In [1]:
import pyaudio
import numpy as np
import matplotlib.pyplot as plt
import wave
import time
from threading import Thread
frames = list()
from librosa import pyin, note_to_hz


def extract(MAX_LEN, RATE, CHUNK):
    global frames

    while True:
        if len(frames) < MAX_LEN:
            continue

        audio_data = np.frombuffer(b''.join(frames), dtype=np.int16)
        frequencies = np.fft.fftfreq(len(audio_data), 1.0 / RATE)
        fft_values = np.fft.fft(audio_data)
        max_index = np.argmax(np.abs(fft_values))
        print(audio_data)
        # Extract the corresponding frequency
        mean_frequency = frequencies[max_index]
        print(f"Mean Frequency: {abs(mean_frequency):.2f} Hz")
        #audio_data = audio_data.astype(np.float32)
        # print(audio_data)
        
        neww = audio_data.astype(np.float32) / np.max(np.abs(audio_data))

        freq0, voiced_flag, voiced_probs = pyin(
            y=neww, 
            fmin=note_to_hz('C2'), 
            fmax=note_to_hz('C7'), 
            sr=RATE, 
            frame_length=4096
        )
        print(freq0)
        # Extract the fundamental frequency
        # freq0 = freq0[voiced_flag]
        # freq0 = np.array(freq0[freq0 > 0])
        # print(f"Fundamental Frequency: {abs(np.mean(freq0)):.2f} Hz")

        time.sleep(1)

def listen(stream, CHUNK, MAX_LEN):
    global frames
    curr_index = 0

    while True:
        data = stream.read(CHUNK)
            
        if len(frames) == MAX_LEN:
            frames[curr_index] = data
            curr_index = (curr_index + 1) % MAX_LEN
        else:
            frames.append(data)
            
        time.sleep(0.1)

def record():
    CHUNK = 1024  # Number of frames per buffer
    FORMAT = pyaudio.paInt16  # Audio format (16-bit PCM)
    CHANNELS = 1  # Number of audio channels (1 for mono, 2 for stereo)
    RATE = 44100  # Sampling rate
    MAX_LEN = 20

    p = pyaudio.PyAudio()
    stream = p.open(
        format=FORMAT,
        channels=CHANNELS,
        rate=RATE,
        input=True,
        frames_per_buffer=CHUNK
    )
    global frames
    print("Recording...")
    t1 = Thread(target = listen, args = [stream, CHUNK, MAX_LEN])
    t2 = Thread(target = extract, args = [MAX_LEN, RATE, CHUNK])

    try:
        t1.start()
        t2.start()

        time.sleep(10)
        t1.join()
        t2.join()

        return
    except KeyboardInterrupt:
        print("Recording stopped by user.")
    finally:
        t1.join()
        t2.join()



record()

Recording...
[ 14  12   6 ... 161 154 149]
Mean Frequency: 1552.54 Hz
Fundamental Frequency: 1579.33 Hz
[ 28  27  30 ... 161 154 149]
Mean Frequency: 370.37 Hz
Fundamental Frequency: 1627.83 Hz
[-605 -639 -664 ... -223 -273 -346]
Mean Frequency: 368.22 Hz


c:\Users\Cody\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Cody\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Fundamental Frequency: nan Hz
[-605 -639 -664 ... -223 -273 -346]
Mean Frequency: 178.73 Hz
Fundamental Frequency: nan Hz
[70 72 71 ... 73 77 76]
Mean Frequency: 176.57 Hz
Fundamental Frequency: nan Hz
[70 72 71 ... 73 77 76]
Mean Frequency: 142.12 Hz
Fundamental Frequency: 144.03 Hz
[  15  134  241 ... -885 -950 -983]
Mean Frequency: 152.89 Hz
Fundamental Frequency: nan Hz
[  15  134  241 ... -885 -950 -983]
Mean Frequency: 516.80 Hz
Fundamental Frequency: nan Hz
[-19  -3   6 ...   7  10   7]
Mean Frequency: 155.04 Hz
Fundamental Frequency: nan Hz
[-19  -3   6 ...   7  10   7]
Mean Frequency: 152.89 Hz
Fundamental Frequency: nan Hz
[-2  0  4 ...  2  4  7]
Mean Frequency: 155.04 Hz
Fundamental Frequency: nan Hz
[  -2    0    4 ... -132 -103  -74]
Mean Frequency: 157.19 Hz
Fundamental Frequency: nan Hz
[-395 -359 -321 ... -132 -103  -74]
Mean Frequency: 284.24 Hz
Fundamental Frequency: nan Hz
[-55 -60 -55 ... -19 -15  -8]
Mean Frequency: 144.27 Hz
Fundamental Frequency: nan Hz
[-55 -60 